Installing necessary packages:
```Julia
using Pkg

Pkg.add(["Random", "Distributions", "DataFrames", "GLM", "Statistics", "ProgressMeter", "Plots"])
```

In [1]:
using Random, Distributions, DataFrames, GLM, Statistics, ProgressMeter, CSV

In [2]:
α = 1.18
β = .4
Q = 20 # Liczba państw
nsims = 10000

N_distr = Poisson(2000)
M_distr = (N, α) -> Poisson.(N .^ α)
p_distr = Uniform(.1,.75)

res = Vector{Any}(missing, nsims)

prog = Progress(10*Threads.nthreads(), "Simulation progress ...")

Threads.@threads for i in 1:(10*Threads.nthreads())
    Threads.threadid()
    next!(prog)
end # end for

Simulation progress ... 100%|████████████████████████████| Time: 0:00:00


In [3]:
prog = Progress(nsims, "Simulation progress ...")

Threads.@threads for k in 1:nsims
    N = reduce(vcat, rand.(Binomial.(rand(N_distr, Q), .1), 1))
    M = reduce(vcat, rand.(M_distr(N, α), 1))
    p = rand(p_distr, Q)
    
    m = reduce(vcat, rand.(Poisson.(M .* p), 1))
    n = reduce(vcat, rand.(Binomial.(N, p), 1))

    df1 = DataFrame(
        y = m,
        x1 = log.(N),
        x2 = log.(n ./ N)
    )

    mm = glm(@formula(y ~ x1 + x2 + 0), df1, Poisson(), LogLink())
    α̂₁, β̂₁ = coef(mm)

    ols = lm(@formula(log(y) ~ x1 + x2 + 0), df1)
    α̂₂, β̂₂ = coef(ols)

    res[k] = [sum(M) sum(N .^ α) sum(N .^ α̂₁) sum(N .^ α̂₂)]
    #push!(res, [sum(M) sum(N .^ α) sum(N .^ α̂₁) sum(N .^ α̂₂)])
    next!(prog)
end # end for

Simulation progress ... 100%|████████████████████████████| Time: 0:00:04


In [4]:
df_res = DataFrame(reduce(vcat, [res[i] for i in 1:length(res) if isassigned(res, i)]), ["actual", "expected", "glm_est", "ols_est"])

describe(df_res)

Row,variable,mean,min,median,max,nmissing,eltype
,Symbol,Float64,Float64,Float64,Float64,Int64,DataType
1,actual,10387.0,9497.0,10384.0,11217.0,0,Float64
2,expected,10387.3,9551.26,10386.7,11088.7,0,Float64
3,glm_est,10123.5,7945.36,10108.0,12671.5,0,Float64
4,ols_est,10040.0,7687.02,10019.4,13291.9,0,Float64


In [5]:
DataFrame(
    rel_bias = [mean((df_res[:, 1] .- df_res[:, 2]) ./ df_res[:, 1]), mean((df_res[:, 1] .- df_res[:, 3]) ./ df_res[:, 1]), mean((df_res[:, 1] .- df_res[:, 4]) ./ df_res[:, 1])],
    rel_mse  = [mean(((df_res[:, 1] .- df_res[:, 2]) .^ 2) ./ df_res[:, 1]), mean(((df_res[:, 1] .- df_res[:, 3]) .^ 2) ./ df_res[:, 1]), mean(((df_res[:, 1] .- df_res[:, 4]) .^ 2) ./ df_res[:, 1])],
    rel_mae  = [mean(abs.(df_res[:, 1] .- df_res[:, 2]) ./ df_res[:, 1]), mean(abs.(df_res[:, 1] .- df_res[:, 3]) ./ df_res[:, 1]), mean(abs.(df_res[:, 1] .- df_res[:, 4]) ./ df_res[:, 1])],
    est      = ["Expected value","glm","ols"]
)

Row,rel_bias,rel_mse,rel_mae,est
,Float64,Float64,Float64,String
1,-0.000124884,0.981482,0.00777724,Expected value
2,0.0253661,33.5443,0.045743,glm
3,0.0334128,49.6033,0.0557511,ols


In [6]:
DataFrame(
    rel_bias = [mean((df_res[:, 2] .- df_res[:, 3]) ./ df_res[:, 2]), mean((df_res[:, 2] .- df_res[:, 4]) ./ df_res[:, 2])],
    rel_mse  = [mean(((df_res[:, 2] .- df_res[:, 3]) .^ 2) ./ df_res[:, 2]), mean(((df_res[:, 2] .- df_res[:, 4]) .^ 2) ./ df_res[:, 2])],
    rel_mae  = [mean(abs.(df_res[:, 2] .- df_res[:, 3]) ./ df_res[:, 2]), mean(abs.(df_res[:, 2] .- df_res[:, 4]) ./ df_res[:, 2])],
    est      = ["glm","ols"]
)

Row,rel_bias,rel_mse,rel_mae,est
,Float64,Float64,Float64,String
1,0.0253979,34.4431,0.0464084,glm
2,0.0334429,50.5185,0.0563489,ols


In [7]:
CSV.write(pwd() * "/data_raw/init_res.csv", df_res);

### Higher population

In [24]:
α = 1.18
β = .4
Q = 7 # Liczba państw
nsims = 10000

N_distr = Poisson(30000)
M_distr = (N, α) -> Poisson.(N .^ α)
p_distr = Uniform(.1,.75)

res = Vector{Any}(missing, nsims)

prog = Progress(nsims, "Simulation progress ...")

Threads.@threads for k in 1:nsims
    N = reduce(vcat, rand.(Binomial.(rand(N_distr, Q), .1), 1))
    M = reduce(vcat, rand.(M_distr(N, α), 1))
    p = rand(p_distr, Q)
    
    m = reduce(vcat, rand.(Poisson.(M .* p), 1))
    n = reduce(vcat, rand.(Binomial.(N, p), 1))

    df1 = DataFrame(
        y = m,
        x1 = log.(N),
        x2 = log.(n ./ N)
    )

    mm = glm(@formula(y ~ x1 + x2 + 0), df1, Poisson(), LogLink())
    α̂₁, β̂₁ = coef(mm)

    ols = lm(@formula(log(y) ~ x1 + x2 + 0), df1)
    α̂₂, β̂₂ = coef(ols)

    res[k] = [sum(M) sum(N .^ α) sum(N .^ α̂₁) sum(N .^ α̂₂)]
    #push!(res, [sum(M) sum(N .^ α) sum(N .^ α̂₁) sum(N .^ α̂₂)])
    next!(prog)
end # end for

df_res = DataFrame(reduce(vcat, [res[i] for i in 1:length(res) if isassigned(res, i)]), ["actual", "expected", "glm_est", "ols_est"])

describe(df_res)

Simulation progress ... 100%|████████████████████████████| Time: 0:00:00


Row,variable,mean,min,median,max,nmissing,eltype
,Symbol,Float64,Float64,Float64,Float64,Int64,DataType
1,actual,88736.7,85713.0,88732.0,91555.0,0,Float64
2,expected,88732.9,85565.2,88728.8,91598.5,0,Float64
3,glm_est,88611.7,76629.4,88608.6,1.02316e5,0,Float64
4,ols_est,88593.8,75254.9,88553.8,1.02756e5,0,Float64


In [25]:
DataFrame(
    rel_bias = [mean((df_res[:, 1] .- df_res[:, 2]) ./ df_res[:, 1]), mean((df_res[:, 1] .- df_res[:, 3]) ./ df_res[:, 1]), mean((df_res[:, 1] .- df_res[:, 4]) ./ df_res[:, 1])],
    rel_mse  = [mean(((df_res[:, 1] .- df_res[:, 2]) .^ 2) ./ df_res[:, 1]), mean(((df_res[:, 1] .- df_res[:, 3]) .^ 2) ./ df_res[:, 1]), mean(((df_res[:, 1] .- df_res[:, 4]) .^ 2) ./ df_res[:, 1])],
    rel_mae  = [mean(abs.(df_res[:, 1] .- df_res[:, 2]) ./ df_res[:, 1]), mean(abs.(df_res[:, 1] .- df_res[:, 3]) ./ df_res[:, 1]), mean(abs.(df_res[:, 1] .- df_res[:, 4]) ./ df_res[:, 1])],
    est      = ["Expected value","glm","ols"]
)

Row,rel_bias,rel_mse,rel_mae,est
,Float64,Float64,Float64,String
1,3.2084e-5,0.98714,0.00265928,Expected value
2,0.00140877,50.8977,0.0182044,glm
3,0.00161109,63.6772,0.0205302,ols


In [26]:
DataFrame(
    rel_bias = [mean((df_res[:, 2] .- df_res[:, 3]) ./ df_res[:, 2]), mean((df_res[:, 2] .- df_res[:, 4]) ./ df_res[:, 2])],
    rel_mse  = [mean(((df_res[:, 2] .- df_res[:, 3]) .^ 2) ./ df_res[:, 2]), mean(((df_res[:, 2] .- df_res[:, 4]) .^ 2) ./ df_res[:, 2])],
    rel_mae  = [mean(abs.(df_res[:, 2] .- df_res[:, 3]) ./ df_res[:, 2]), mean(abs.(df_res[:, 2] .- df_res[:, 4]) ./ df_res[:, 2])],
    est      = ["glm","ols"]
)

Row,rel_bias,rel_mse,rel_mae,est
,Float64,Float64,Float64,String
1,0.00136558,51.8865,0.0184015,glm
2,0.00156737,64.7593,0.0206959,ols


In [27]:
CSV.write(pwd() * "/data_raw/init_res_1.csv", df_res);